# Отчет: задание 2

*Неверо Андрей, лабораторная работа 2, 2 курс, 4 группа*

В задании требовалось написать программу, которая вычисляет характеристический многочлен матрицы с помощью метода Данилевского, находит его действительные корни, после чего восстанавливает собственные вектора, соответствующие найденным собственным значениям.

Решением данной задачи можно считать реализацию методов *CountFrobeniusMatrix*, *CountCharacteristicPolynomial*, *FindCharacteristicPolynomialRoots*, *GetEigenvectorsFromFrobeniusMatrix* класса Matrix, а также методов *RunBisectionAlgorithm*, *RunNewtonAlgorithm*, *FindRoots* класса Polynomial (см. соответствующие заголовочные файлы).

Обработка связанных с этим заданием данных вынесена в отдельный класс, определенный в заголовочном файле *Task7*.

Реализованный алгоритм позволяет вычислить характеристический многочлен любой матрицы. Если многочлен пригоден для использования методов бисекции и Ньютона, алгоритм может найти все его действительные корни, которые будут являться собственными значениями матрицы. Если матрица приводится к обычной (не блочной) форме Фробениуса, алгоритм также может восстановить соответствующие действительным собственным значениям собственные вектора.

## Описание алгоритма нахождения характеристического многочлена

Используется распространенная редакция метода Данилевского. Матрица приводится к форме Фробениуса с помощью преобразований подобия. При необходимости столбцы матрицы меняются местами (происходит домножение на матрицы перестановок). Если даже перемена мест столбцов не позволяет привести матрицу к канонической форме Фробениуса (не блочной), используется блочная форма Фробениуса, которая позволяет вычислить характеристический многочлен, но не позволяет в дальнейшем восстановить собственные вектора.

В процессе приведения матрицы к форме Фробениуса сохраняется информация об используемых матрицах преобразований. Она необходима для восстановления собственных векторов.

## Описание алгоритма нахождения корней многочлена

Процесс отделения корней реализован рекурсивно:
* Для многочленов первой и второй степени корни находятся в явном виде.
* Для многочленов третьей и более высокой степени в качестве отрезков, потенциально содержащих корни, принимаются отрезки между корнями производной многочлена (а как известно, производная имеет меньшую степень).

В качестве базового отрезка, на котором осуществляется поиск корней, взят отрезок от минус до плюс миллиарда (при необходимости эти границы можно расширить или сузить). Этот отрезок делится на некоторое количество более мелких (если производная имеет корни), после чего работа проводится уже отдельно над каждым отрезком из множества.

Сначала любой отрезок сужается до размеров $\approx 0.01$ с помощью метода бисекции. Применение метода Ньютона на данном этапе выглядит достаточно рискованным, так как в определенных случаях (для некоторых многочленов) он наверняка не выдаст корректный результат для исходных больших отрезков. Более того, даже в нашем случае, если корни многочлена отличаются менее, чем на $0.01$, алгоритму может не удастся отыскать их все. При необходимости можно уменьшить размер отрезка, до которого работает метод бисекции, и найти эти корни.

После сужения для каждого отрезка запускается метод Ньютона. Сходимость метода изначально проверяется с помощью условия Фурье. Итерации продолжаются до тех пор, пока не будет достигнута требуемая точность

Описанный подход позволяет найти действительные корни многочлена небольшой степени (не берем в рассчет многочлены, на которых метод Ньютона не даст корректный результат даже для выбранного относительно малого отрезка). Для многочленов большой степени применение данных методов выглядит не очень перспективным в связи с большим количеством рекурсивных вычислений.

Отдельно стоит обратить внимание на случаи кратных корней. Реализованный алгоритм вернет любой кратный корень как минимум один раз, но в общем случае любой такой корень может содержаться в результатах любое количество раз от 1 до его кратности (хотя некоторые меры для снижения количества дубликатов в результатах были приняты).

## Описание алгоритма нахождения собственных векторов

Как известно, если $\lambda$ - собственное значение матрицы Фробениуса, то одним из собственных векторов, соответствующих этому значению, является вектор вида:

$$ y = \begin{bmatrix}
    \lambda^{n - 1} \\
    \lambda^{n - 2} \\
    \ldots \\
    \lambda \\
    1
\end{bmatrix} $$

При этом если $y$ является собственным значением матрицы Фробениуса $\Phi$, а данная матрица получается из $A$ как $\Phi = S^{-1} A S$, то собственным вектором матрицы $A$ будет являться вектор $Sy$ (причем соответствовать он будет тому же собственному значению). Данный алгоритм работает для матриц Фробениуса в каноническом (не блочном) виде, он и используется для восстановления собственных векторов.

Понятно, что в случае, когда подпространство собственных векторов, соответствующее некоторому собственному значению, является более чем одномерным, мы сможем вернуть только один собственный вектор.

В перспективе можно реализовать нахождение собственных векторов для блочных матриц Фробениуса с помощью решения однородных систем уравнений.

## Пункт 1

Характеристический многочлен матрицы имеет вид:

$$ \lambda^{10} + 16\lambda^{8} + 8\lambda^{7} - 26\lambda^{6} + 193\lambda^{5} - 2530\lambda^{4} - 1304\lambda^{3} - 2347\lambda^{2} + 5519\lambda + 436\lambda $$

Его действительные корни:

$$ \lambda \in \{-3.41571, -0.0765952, 0.997202, 3.11316\} $$

Матрица Фробениуса, подобная исходной, не является блочной, поэтому возможно корректное вычисление собственных векторов. Приведем их:

In [1]:
from data import task7_1

for i in range (len(task7_1.values)):
    print(task7_1.values[i])
    print(task7_1.vectors[i])

-3.41571
[[ 1.12023 ]
 [ 0.244605]
 [ 0.547816]
 [ 0.851135]
 [-0.194022]
 [ 0.321677]
 [ 0.842733]
 [-0.917718]
 [-0.666671]
 [ 1.      ]]
-0.0765952
[[-0.364601]
 [ 1.20625 ]
 [-1.08266 ]
 [-0.720419]
 [ 0.629124]
 [ 1.54224 ]
 [-0.579526]
 [ 1.90776 ]
 [ 1.97713 ]
 [ 1.      ]]
0.997202
[[ 2.1507  ]
 [-0.933021]
 [-0.126631]
 [-2.01484 ]
 [ 1.92807 ]
 [ 0.421333]
 [ 0.431789]
 [ 1.31244 ]
 [-0.402388]
 [ 1.      ]]
3.11316
[[-5.19828]
 [ 7.29669]
 [ 6.79538]
 [ 5.82337]
 [ 6.2476 ]
 [-4.79443]
 [-7.30554]
 [-2.37793]
 [ 7.26184]
 [ 1.     ]]


## Пункт 2

Характеристический многочлен матрицы имеет вид:

$$ x^{20} - 6x^{18} - 9x^{17} - 149x^{16} + 22x^{15} - 667x^{14} + 1571x^{13} + 9889x^{12} + 6871x^{11} + 104968x^{10} - 186788x^{9} - 48050x^{8} - 789911.000001x^{7} - 2150249.000001x^{6} + 7583712.999994x^{5} - 6198074.999978x^{4} - 2624315.000043x^{3} + 9185950.000009x^{2} + 3734910.000030x + 1364637.000002 $$

$$ x \in \{-3.66921, -3.23279, -2.29641, -0.813381, 1.6423, 2.30574, 2.99193, 3.85232\} $$

Матрица Фробениуса, подобная исходной, снова не является блочной, поэтому возможно корректное вычисление собственных векторов. Приведем их.

In [3]:
from data import task7_2

for i in range (len(task7_2.values)):
    print(task7_2.values[i])
    print(task7_2.vectors[i])

-3.66921
[[-2.41494 ]
 [-1.28626 ]
 [ 1.86053 ]
 [-2.031   ]
 [ 2.67023 ]
 [ 0.525332]
 [-2.93361 ]
 [-1.63566 ]
 [ 0.202241]
 [-2.29067 ]
 [-1.1846  ]
 [ 2.9538  ]
 [-3.47075 ]
 [ 2.23619 ]
 [-1.17749 ]
 [ 0.70586 ]
 [ 3.14679 ]
 [ 3.62711 ]
 [-2.05491 ]
 [ 1.      ]]
-3.23279
[[ 1.94452 ]
 [-0.154877]
 [-1.5558  ]
 [ 1.95998 ]
 [-3.30176 ]
 [-0.248779]
 [ 4.04717 ]
 [ 1.05627 ]
 [-1.50744 ]
 [-0.34831 ]
 [-0.532879]
 [-1.90055 ]
 [ 2.6889  ]
 [-2.17341 ]
 [ 2.98452 ]
 [-1.62186 ]
 [-2.20641 ]
 [-4.08848 ]
 [ 3.36227 ]
 [ 1.      ]]
-2.29641
[[-0.541812 ]
 [ 1.24189  ]
 [ 0.295173 ]
 [-1.31761  ]
 [-0.218461 ]
 [-0.113351 ]
 [-0.772612 ]
 [ 0.989326 ]
 [ 0.201788 ]
 [ 0.605188 ]
 [ 0.678589 ]
 [-0.337911 ]
 [ 0.0359236]
 [-0.908809 ]
 [-0.0553363]
 [ 0.225162 ]
 [-1.21939  ]
 [-0.489876 ]
 [-0.505219 ]
 [ 1.       ]]
-0.813381
[[-1.20537 ]
 [ 1.86708 ]
 [-0.246464]
 [ 0.257385]
 [ 0.539093]
 [ 1.15935 ]
 [-0.958357]
 [-0.195384]
 [-2.01399 ]
 [ 0.167293]
 [-0.204013]
 [ 1.6182  ]
 [ 1